In [1]:
import torch
import torch.nn.functional as F
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device =", device)

device = cuda


In [2]:
model_dir = "./kmbert_epoch1"   # 노트북이랑 같은 폴더라면 이대로 쓰면 됨

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir).to(device)

id2label = model.config.id2label
label2id = model.config.label2id

print("num_labels:", len(id2label))
print("예시 라벨 5개:", [id2label[i] for i in range(5)])

num_labels: 583
예시 라벨 5개: ['B형간염', 'C형간염', 'HIV 감염', 'VDT 증후군', '가와사키병']


In [3]:
# TODO: 필요하면 여기 경로만 너 환경에 맞게 수정!
csv_path = r"C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\mediq_labels_final.csv"

df_label = pd.read_csv(csv_path)
print("shape:", df_label.shape)
print(df_label.head())
print("columns:", df_label.columns.tolist())

shape: (583, 5)
     label department severity  reviewed comment
0     B형간염      소화기내과       주의         0     NaN
1     C형간염      소화기내과       주의         0     NaN
2   HIV 감염       감염내과       위험         0     NaN
3  VDT 증후군       신경외과       일반         0     NaN
4    가와사키병     소아청소년과       위험         0     NaN
columns: ['label', 'department', 'severity', 'reviewed', 'comment']


In [4]:
# df_label 컬럼명이 다르면 여기 이름만 수정해주면 됨
LABEL_COL = "label"
DEPT_COL = "department"
SEV_COL = "severity"

LABEL2DEPT = {}
LABEL2SEVERITY = {}

for _, row in df_label.iterrows():
    lbl = str(row[LABEL_COL]).strip()
    dept = str(row[DEPT_COL]).strip()
    sev  = str(row[SEV_COL]).strip()
    LABEL2DEPT[lbl] = dept
    LABEL2SEVERITY[lbl] = sev

print("라벨 수:", len(LABEL2DEPT))
list(LABEL2DEPT.items())[:10]

라벨 수: 583


[('B형간염', '소화기내과'),
 ('C형간염', '소화기내과'),
 ('HIV 감염', '감염내과'),
 ('VDT 증후군', '신경외과'),
 ('가와사키병', '소아청소년과'),
 ('각막염', '안과'),
 ('간 전이', '소화기내과'),
 ('간경화', '소화기내과'),
 ('간농양', '소화기내과'),
 ('간성 뇌증', '소화기내과')]

In [5]:
import pandas as pd

# 1) 응급/중증 → 서비스 대상 아님 (OUT_OF_SCOPE)
OUT_OF_SCOPE_KEYWORDS = [
    # ① 호흡/의식 관련
    "호흡곤란", "숨이 안 쉬어", "숨이 막혀", "숨 막혀", "질식할 것 같아",
    "의식 잃음", "기절", "쓰러짐", "정신이 흐려짐",

    # ② 마비/신경학적 이상
    "마비", "반신마비", "몸이 안 움직여", "팔 다리가 안 움직여",
    "얼굴 마비", "입이 돌아감", "혀가 돌아감", "말이 안 나와", "발음이 새어",

    # ③ 시력/시야 이상
    "갑자기 안 보임", "시야가 안 보여", "시야가 흐려짐", "갑자기 시력 저하",

    # ④ 가슴/심장 관련
    "가슴이 쥐어짜는", "가슴이 조여", "가슴이 찢어질 듯 아파",
    "심장이 쿵쿵거려서 숨이 찬", "갑작스러운 흉통",

    # ⑤ 출혈/혈변/토혈
    "피를 토함", "토혈", "피 섞인 토", "혈변", "검은 변", "새까만 변", "대량 출혈",
    "코피가 멈추지 않음", "피가 멈추지 않아",

    # ⑥ 외상/사고
    "교통사고", "머리를 세게 부딪힘", "머리를 세게 부딪쳤음",
    "높은 곳에서 떨어짐", "골절 의심", "뼈가 보임",

    # ⑦ 복부 응급
    "참을 수 없는 복통", "찢어질 듯한 복통",
    "복부가 단단해짐", "배가 단단해짐", "복통에 식은땀",

    # ⑧ 고열 + 전신 상태 악화
    "40도", "40 도", "열이 40", "고열에 떨림",
    "열이 안 내려감", "해열제를 먹어도 열이 안 떨어짐",
    "경련", "발작"
]

# 2) 경증/만성이지만 '주의'가 필요한 표현 (RISK = MEDIUM)
WARNING_KEYWORDS = [
    "1주 이상", "일주일 이상", "2주 이상", "한 달 이상",
    "지속되는", "계속 아픔", "계속 아파요",
    "반복되는", "자주 반복", "자주 재발",
    "밤에 계속", "밤마다", "자다가 깨는",
    "먹으면 바로 아픔", "먹으면 바로 아파요",
    "저림이 계속", "저린 게 계속", "목이 뻣뻣함", "목이 뻣뻣해요"
]

# 3) 경증 질환 (Light Disease) 예시 라벨용 (필요하면 CSV 따로 만들 수도 있음)
LIGHT_DISEASE_TERMS = [
    "두통", "긴장성 두통", "VDT 증후군",
    "감기", "콧물", "기침", "인후통",
    "소화불량", "가벼운 복통", "설사", "변비",
    "근육통", "어깨 결림", "피로", "피로감",
    "메스꺼움", "오심", "숙취"
]

# 4) 만성/진행성 질환 (Slow-progressing) 예시
SLOW_PROGRESSING_TERMS = [
    "역류성 식도염", "만성 위염", "기능성 소화불량",
    "만성편두통", "편두통",
    "거북목 증후군", "목 디스크", "허리 디스크",
    "안구건조증", "VDT 증후군",
    "아토피 피부염", "습진", "만성 비염",
    "과민성 장 증후군"
]

In [6]:
rules = []

# out_of_scope 규칙
for kw in OUT_OF_SCOPE_KEYWORDS:
    rules.append({
        "category": "out_of_scope",
        "keyword": kw
    })

# warning 규칙
for kw in WARNING_KEYWORDS:
    rules.append({
        "category": "warning",
        "keyword": kw
    })

# light disease 용 키워드 (원하면 생략 가능)
for kw in LIGHT_DISEASE_TERMS:
    rules.append({
        "category": "light_term",
        "keyword": kw
    })

# slow-progressing 용 키워드
for kw in SLOW_PROGRESSING_TERMS:
    rules.append({
        "category": "slow_progressing_term",
        "keyword": kw
    })

# 판다스로 DataFrame 만들기
df_rules = pd.DataFrame(rules)

# 📌 여기만 네 PC 경로로 수정!
df_rules.to_csv(
    r"C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\mediq_scope_rules.csv",
    index=False,
    encoding="utf-8-sig"
)

df_rules.head()

,category,keyword
0,out_of_scope,호흡곤란
1,out_of_scope,숨이 안 쉬어
2,out_of_scope,숨이 막혀
3,out_of_scope,숨 막혀
4,out_of_scope,질식할 것 같아


In [7]:
# 규칙 CSV 로드
rules_path = r"C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\mediq_scope_rules.csv"
df_rules = pd.read_csv(rules_path)
print(df_rules.head())
print(df_rules["category"].value_counts())

       category   keyword
0  out_of_scope      호흡곤란
1  out_of_scope   숨이 안 쉬어
2  out_of_scope     숨이 막혀
3  out_of_scope      숨 막혀
4  out_of_scope  질식할 것 같아
category
out_of_scope             55
warning                  19
light_term               18
slow_progressing_term    14
Name: count, dtype: int64


In [8]:
def apply_text_scope_filter(text: str, df_rules: pd.DataFrame):
    text = str(text)
    
    # 카테고리별 매칭 결과
    matches = {
        "out_of_scope": [],
        "warning": [],
        "light_term": [],
        "slow_progressing_term": [],
    }

    # 카테고리별 키워드 매칭
    for cat in df_rules["category"].unique():
        kws = df_rules[df_rules["category"] == cat]["keyword"].dropna().tolist()
        for kw in kws:
            if kw and kw in text:
                matches.setdefault(cat, []).append(kw)

    # 최종 스코프 결정 로직
    if matches["out_of_scope"]:
        scope = "OUT_OF_SCOPE"   # 메디큐 서비스 대상 아님(응급/중증)
    elif matches["warning"]:
        scope = "WARNING"        # 경증/만성이지만 병원 권고
    else:
        scope = "SAFE"           # 텍스트 기준으로는 문제 없음

    return {
        "scope": scope,
        "matches": matches,
    }

In [9]:
tests = [
    "갑자기 가슴이 조여오고 숨이 막혀요",           # OUT_OF_SCOPE 나와야 함
    "두통이 1주 이상 계속 이어지고 있어요",          # WARNING
    "요즘 배가 살살 아프고 설사를 몇 번 했어요",     # SAFE + GI 관련
]

for t in tests:
    res = apply_text_scope_filter(t, df_rules)
    print("문장:", t)
    print(" → scope:", res["scope"])
    print(" → matches:", res["matches"])
    print("-" * 60)

문장: 갑자기 가슴이 조여오고 숨이 막혀요
 → scope: OUT_OF_SCOPE
 → matches: {'out_of_scope': ['숨이 막혀', '가슴이 조여'], 'warning': [], 'light_term': [], 'slow_progressing_term': []}
------------------------------------------------------------
문장: 두통이 1주 이상 계속 이어지고 있어요
 → scope: WARNING
 → matches: {'out_of_scope': [], 'warning': ['1주 이상'], 'light_term': ['두통'], 'slow_progressing_term': []}
------------------------------------------------------------
문장: 요즘 배가 살살 아프고 설사를 몇 번 했어요
 → scope: SAFE
 → matches: {'out_of_scope': [], 'warning': [], 'light_term': ['설사'], 'slow_progressing_term': []}
------------------------------------------------------------


In [10]:
import torch
import torch.nn.functional as F

def predict_label_kmbert(text: str, k: int = 5):
    """
    텍스트 1개를 넣으면
    (라벨, 확률) 튜플 리스트를 top-k까지 반환
    예: [("두통", 0.82), ("긴장성 두통", 0.07), ...]
    """
    enc = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True
    ).to(model.device)

    with torch.no_grad():
        logits = model(**enc).logits      # [1, num_labels]
        probs  = F.softmax(logits, dim=-1)[0]  # [num_labels]

    topk = torch.topk(probs, k)
    top_indices = topk.indices.tolist()
    top_probs   = topk.values.tolist()
    top_labels  = [id2label[i] for i in top_indices]

    return list(zip(top_labels, [float(p) for p in top_probs]))

In [11]:
# GI 문맥(배탈/장염) 여부 판단용 키워드
GI_CONTEXT_KEYWORDS = [
    "배가", "배 아파", "배가 아파", "복통",
    "속이", "속이 안 좋", "설사", "설사를",
    "묽은 변", "메스꺼", "구역", "구토", "토했", "체했",
]

# "배탈 느낌"일 때 선택해도 괜찮은 비교적 가벼운 GI 라벨들
GI_SAFE_LABELS = {
    "급성 위장염",
    "기능성 위장 장애",
    "노로바이러스 장염",
    "바이러스성 장염",
    "복부 통증",
    "식중독",
}

# "배탈 느낌 문장"에는 너무 무겁거나 안 어울리는 GI 라벨들
GI_BAD_LABELS = {
    "복수",
    "복막염",
    "췌장염",
    "만성 췌장염",
    "췌장암",
    "소장암",
    "조기 위암",
    "위선종",
    "식도암",
    "담관암",
    "간세포성 암종",
    "간암",
}

In [12]:
def mediq_predict(text: str, k: int = 5):
    """
    MediQ 최종 엔진:
    1) 텍스트 범위 필터 (OUT_OF_SCOPE / WARNING / SAFE)
    2) SAFE/WARNING 이면 KM-BERT로 증상 정규화
    3) 라벨에 대응하는 진료과 / severity 함께 반환
    """
    # 1) 텍스트 기반 범위 필터
    scope_res = apply_text_scope_filter(text, df_rules)
    scope = scope_res["scope"]

    # OUT_OF_SCOPE면 정규화/진료과 안내 하지 않음
    if scope == "OUT_OF_SCOPE":
        return {
            "mode": "out_of_scope",
            "input_text": text,
            "scope": scope_res,
            "message": "응급/중증 가능성이 있어 MediQ 서비스 대상이 아닙니다. 즉시 응급실 또는 119를 이용해주세요."
        }

    # 2) 정규화 모델 실행
    topk = predict_label_kmbert(text, k=k)
    final_label, final_prob = topk[0]

    # 3) 라벨 → 진료과 / severity 매핑
    dept = LABEL2DEPT.get(final_label, None)
    sev  = LABEL2SEVERITY.get(final_label, None)

    # ---------------------------------------------------
    # 🔹 GI(소화기) 안전 규칙
    #  - 문장에 배/설사/구토 등 GI 문맥이 있고
    #  - (1) 최종 라벨이 GI_BAD_LABELS 안에 있거나
    #    (2) 진료과가 소화기내과인데 severity가 너무 무거운 경우(위험/응급)
    #    → top-k 안에서 GI_SAFE_LABELS 중 하나로 교체 시도
    # ---------------------------------------------------
    contains_gi_kw = any(kw in text for kw in GI_CONTEXT_KEYWORDS)

    if contains_gi_kw and (
        final_label in GI_BAD_LABELS or
        (dept == "소화기내과" and sev in ["위험", "응급"])
    ):
        replaced = False

        # 1차: top-k 안에서 안전 GI 라벨 찾기
        for lbl, prob in topk:
            if lbl in GI_SAFE_LABELS:
                final_label = lbl
                final_prob = prob
                dept = LABEL2DEPT.get(final_label, dept)
                sev  = LABEL2SEVERITY.get(final_label, sev)
                replaced = True
                break

        # 2차: top-k에 없으면, 미리 정한 우선순위 순서로 fallback
        if not replaced:
            for cand in ["급성 위장염", "바이러스성 장염", "노로바이러스 장염", "복부 통증"]:
                if cand in LABEL2DEPT:
                    final_label = cand
                    # 확률은 그냥 기존 값 유지
                    dept = LABEL2DEPT[cand]
                    sev  = LABEL2SEVERITY[cand]
                    replaced = True
                    break
        # (replaced 안 됐다면 그대로 두지만, 이 경우는 거의 없을 것)
    # ---------------------------------------------------

    return {
        "mode": "normal",            # 또는 "warning" 등을 나중에 세분화 가능
        "input_text": text,
        "scope": scope_res,          # SAFE / WARNING + 어떤 키워드 잡혔는지
        "final_label": final_label,  # 정규화된 증상 라벨
        "final_prob": final_prob,    # 그 라벨의 확률
        "department": dept,          # 진료과
        "severity": sev,             # 라벨 CSV에 있는 severity(일반/주의/위험/응급)
        "topk": topk,                # top-k 결과 전체
    }

In [13]:
examples = [
    "갑자기 가슴이 조여오고 숨이 막혀요",            # OUT_OF_SCOPE
    "두통이 1주 이상 계속 이어지고 있어요",           # WARNING + 두통 계열 라벨
    "요즘 배가 살살 아프고 설사를 몇 번 했어요",      # SAFE + GI 라벨
]

for t in examples:
    res = mediq_predict(t, k=5)
    print("문장:", t)
    print("mode:", res["mode"])
    print("scope:", res["scope"]["scope"], "| matches:", res["scope"]["matches"])
    
    if res["mode"] == "normal":
        print(" → 정규화 라벨:", res["final_label"], f"({res['final_prob']:.3f})")
        print(" → 진료과:", res["department"], "| severity:", res["severity"])
        print(" → top5:", res["topk"])
    else:
        print(" → 메시지:", res["message"])
    print("-" * 80)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


문장: 갑자기 가슴이 조여오고 숨이 막혀요
mode: out_of_scope
scope: OUT_OF_SCOPE | matches: {'out_of_scope': ['숨이 막혀', '가슴이 조여'], 'warning': [], 'light_term': [], 'slow_progressing_term': []}
 → 메시지: 응급/중증 가능성이 있어 MediQ 서비스 대상이 아닙니다. 즉시 응급실 또는 119를 이용해주세요.
--------------------------------------------------------------------------------
문장: 두통이 1주 이상 계속 이어지고 있어요
mode: normal
scope: WARNING | matches: {'out_of_scope': [], 'warning': ['1주 이상'], 'light_term': ['두통'], 'slow_progressing_term': []}
 → 정규화 라벨: 두통 (0.991)
 → 진료과: 신경과 | severity: 주의
 → top5: [('두통', 0.9912444353103638), ('편두통', 0.007589436136186123), ('이명', 0.00013767735799774528), ('발작', 0.00010000043403124437), ('냉방병', 9.367556049255654e-05)]
--------------------------------------------------------------------------------
문장: 요즘 배가 살살 아프고 설사를 몇 번 했어요
mode: normal
scope: SAFE | matches: {'out_of_scope': [], 'warning': [], 'light_term': ['설사'], 'slow_progressing_term': []}
 → 정규화 라벨: 급성 위장염 (0.836)
 → 진료과: 소화기내과 | severity: 일반
 → top5: [('복수', 0.

In [14]:
res = mediq_predict("누우면 속이 불편해지는 것 같아요.", k=5)
print("scope:", res["scope"]["scope"])
print("정규화 라벨:", res["final_label"], res["final_prob"])
print("진료과:", res["department"], "severity:", res["severity"])
print("top5:", res["topk"])

scope: SAFE
정규화 라벨: 욕창 0.9924345016479492
진료과: 성형외과 severity: 주의
top5: [('욕창', 0.9924345016479492), ('복수', 0.0011722956551238894), ('복막염', 0.0008967227186076343), ('봉와직염', 0.00047526127309538424), ('화상', 0.00026701088063418865)]
